#Predição de Evasão na Indústria de Telefonia*

*Dados adaptados de `https://www.kaggle.com/code/bandiatindra/telecom-churn-prediction/data` com traduções e algum pré-processamento

####A rotatividade de clientes é um dos maiores problemas da indústria de telecomunicações. A pesquisa mostrou a taxa média de rotatividade mensal entre as 4 principais operadoras de telefonia móvel nos EUA é de 1,9% a 2%.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

####Leia o arquivo de dados no notebook python

In [ ]:
telecom_cust = pd.read_csv('4th semester/data_science/2023-09-05/telecom_churn_trad.csv')

####Dê uma espiada nas primeiras cinco observações

In [ ]:
telecom_cust.head()

####Liste os nomes das features (colunas)

In [ ]:
telecom_cust.columns.values

####Explore os dados para ver se há algum valor ausente.

####Começe olhando os tipos de dados das features.

In [ ]:
telecom_cust.dtypes

####Qual o tipo de `CobrançaTotal`?

####Converta `CobrançaTotal` para um valor numérico.

In [ ]:
telecom_cust.CobrançaTotal = pd.to_numeric(telecom_cust.CobrançaTotal, errors='coerce')

####Agora verifique os valores ausentes.

Quantos valores estão ausentes?

In [ ]:
telecom_cust.isnull().sum()

####Remova as linhas com valores ausentes.

*Obs: Essa é a **pior** abordagem possível. Porque?*

In [ ]:
telecom_cust.dropna(inplace=True)

####Verifique os valores de `Evasão`

Como os valores estão distribuídos?

In [ ]:
telecom_cust.Evasão.value_counts()

####Converta `Evasão` em uma variável numérica binária

In [ ]:
telecom_cust['Evasão'].replace(to_replace='Sim', value=1, inplace=True)
telecom_cust['Evasão'].replace(to_replace='Não',  value=0, inplace=True)

#### Realize a Codificação One Hot (One Hot Encoding) para transformar as variáveis categóricas em features (colunas).

Obs: Faça isso com todas as variáveis categóricas, excluindo `IDCliente`

In [ ]:
df_dummies = pd.get_dummies(telecom_cust.drop('IDCliente', axis=1))

####Dê uma espiada em como ficarm os dados

In [ ]:
df_dummies.head()

#### Agora que consegue fazer a análise, investigue a correlação de `Evasão` com as outras features

In [ ]:
df_dummies.corr()['Evasão'].sort_values(ascending = False)

#### Plote um gráfico da correlação para melhor visualização

In [ ]:
plt.figure(figsize=(15,8))
df_dummies.corr()['Evasão'].sort_values(ascending = False).plot(kind='bar')

####`Contrato_Mensal`, ausência de `SegurançaOnline` e `SuporteTécnico` estão positivamente correlacionados evasão. Em contrapartida, `TempoContrato` longo, ter `Contrato` de `Dois anos` e `Sem serviço de internet` estão negativamente correlacionados com a evasão.

#### Curiosamente, serviços como `SegurançaOnline`, `StreamingTV`, `BackupOnline`, `SuporteTécnico`, etc. "sem conexão com a Internet" parecem estar negativamente correlacionados com a evaão.

#### Explore os padrões das correlações acima antes de se aprofundar na modelagem e na identificação das variáveis importantes para seu modelo.

## Rápida EDA (Análise Exploratória de Dados)

Comece primeiro explorando o conjunto de dados para compreender melhor os padrões nos dados e, potencialmente, formar algumas hipóteses. Comece examinando a distribuição de variáveis individuais e, em seguida, vasculhe os dados em busca de quaisquer tendências interessantes.

### 1. Demografia - Comece tentando entender o sexo, faixa etária, cônjuge e dependentes dos clientes


#### Qual a porcentagem de cliente Feminino e Masculino?

In [ ]:
colors = ['#4D3425','#E4512B']
ax = (telecom_cust['Gênero'].value_counts()*100.0 /len(telecom_cust)).plot(kind='bar')
ax.set_ylabel('% Clientes')
ax.set_xlabel('Gênero')
ax.set_ylabel('% Clientes')
ax.set_title('Distribuição de Gênero')

# Para decorar a barra da esquerda e colocar rótulos nas barras
# crie uma lista para coletar os dados de plt.patches
totals = []

# encontre os valores e acrescente à lista
for i in ax.patches:
    totals.append(i.get_width())

# defina rótulos de barras individuais usando a lista acima
total = sum(totals)

for i in ax.patches:
    # get_width empurra para a esquerda ou para a direita;
    # get_y empurra para cima ou para baixo
    ax.text(i.get_x()+.15, i.get_height()-3.5, \
            str(round((i.get_height()/total), 1))+'%',
            fontsize=12,
            color='white',
           weight = 'bold')


### Qual é o gasto máximo e a média de gasto mensal entre os clientes do sexo feminino e masculino?

In [ ]:
telecom_cust.groupby('Gênero')['CobrançaMensal'].agg(['mean', 'max'])

### Quantos % são idosos?

In [ ]:
ax = (telecom_cust['Idoso'].value_counts()*100.0 /len(telecom_cust))\
.plot.pie(autopct='%.1f%%', labels = ['Não', 'Sim'],figsize =(5,5), fontsize = 12 )
ax.set_ylabel('Idosos',fontsize = 12)
ax.set_title('% de Idosos', fontsize = 12);

### Qual é o gasto máximo e a média de gasto mensal entre os clientes idosos?

In [ ]:
telecom_cust.groupby('Idoso')['CobrançaMensal'].agg(['mean', 'max', 'median'])

### Quantos clientes tem cônjuge e quantos têm dependente?

In [ ]:
df2 = pd.melt(telecom_cust, id_vars=['IDCliente'], value_vars=['Dependentes','Cônjuje'])
df3 = df2.groupby(['variable','value']).count().unstack()
df3 = df3*100/len(telecom_cust)

ax = df3.loc[:,'IDCliente'].plot.bar(stacked=True,
                                      figsize=(8,6),rot = 0,
                                     width = 0.2)

ax.set_ylabel('% Clientes',size = 14)
ax.set_xlabel('')
ax.set_title('% Clientes com Dependentes e Cônjujes',size = 14)
ax.legend(loc = 'center',prop={'size':14})

### Quem gasta mais entre os clientes com  com Conjujes e clientes sem Conjujes?

In [ ]:
telecom_cust.groupby('Cônjuje')['CobrançaMensal'].agg(['mean', 'max', 'median'])

## 2. TempoContrato e tipo de Contrato

### Obtenha a distribuição de clientes por `TempoContrato` e observe os resultados.

In [ ]:
ax = sns.histplot(telecom_cust['TempoContrato'], bins=int(180/5))
ax.set_ylabel('# dos Clientes')
ax.set_xlabel('TempoContrato (meses)')
ax.set_title('# de Clientes pelo seu TempoContrato')

### Qual o tipo de contrato preferido pelos assinantes?

In [ ]:
ax = telecom_cust['Contrato'].value_counts().plot(kind = 'bar',rot = 0, width = 0.3)
ax.set_ylabel('# de Clientes')
ax.set_title('# de Clientes por tipo de Contrato');

### Como o TempoContrato depende do tipo de Contrato?

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, sharey = True, figsize = (20,6))

ax = sns.histplot(telecom_cust[telecom_cust['Contrato']=='Mensal']['TempoContrato'],
                   bins=int(180/5), color = 'turquoise',
                 ax=ax1)
ax.set_ylabel('# de Clientes')
ax.set_xlabel('TempoContrato (meses)')
ax.set_title('Contrato Mensal')

ax = sns.histplot(telecom_cust[telecom_cust['Contrato']=='Um ano']['TempoContrato'],
                   bins=int(180/5), color = 'steelblue',
                 ax=ax2)
ax.set_xlabel('TempoContrato (meses)')
ax.set_title('Contrato de um ano')

ax = sns.histplot(telecom_cust[telecom_cust['Contrato']=='Dois anos']['TempoContrato'],
                   bins=int(180/5), color = 'darkblue',
                 ax=ax3)

ax.set_xlabel('TempoContrato (meses)')
ax.set_title('Contrato de dois anos');


Os clientes do Contrato de 2 anos são mais leais: a maioria dos clientes mensais usa serviços por 1-2 meses no total, enquanto a maioria dos clientes do Contrato de 2 anos usa serviços por mais de 50 meses (> 4 anos).

## 3. E quanto aos serviços adicionais?

In [ ]:
servicos = ['LinhaTelefone','MúltiplasLinhas','ServicoInternet','SegurançaOnline',
           'BackupOnline','ProteçãoDispositivos','SuporteTecnico','StreamingTV','StreamingFilmes']

fig, axes = plt.subplots(nrows = 3,ncols = 3,figsize = (15,12))
for i, item in enumerate(servicos):
    if i < 3:
        ax = telecom_cust[item].value_counts().plot(kind = 'bar',ax=axes[i,0],rot = 0)

    elif i >=3 and i < 6:
        ax = telecom_cust[item].value_counts().plot(kind = 'bar',ax=axes[i-3,1],rot = 0)

    elif i < 9:
        ax = telecom_cust[item].value_counts().plot(kind = 'bar',ax=axes[i-6,2],rot = 0)
    ax.set_title(item)


### Existe diferença entre clientes idosos ou não, em relação à `ServicoInternet`?

In [ ]:
telecom_cust.groupby('Idoso')['ServicoInternet'].value_counts() * 100/len(telecom_cust)

### Observamos a mesma tendência para `SuporteTecnico`?

In [ ]:
telecom_cust.groupby('Idoso')['SuporteTecnico'].value_counts()

# 4. E quanto à evasão (o que queremos analisar)?

### Qual percentual de clientes não evadem qual percentual que evadem?

In [ ]:
telecom_cust['Evasão'].value_counts() / len(telecom_cust) * 100

### Qual a média de `TempoContrato` entre os clientes que evadem e não evadem?

In [ ]:
telecom_cust.groupby('Evasão').TempoContrato.agg(['mean', 'median', 'std'])

### Qual a média de evasão por tipo de `Contrato`?

In [ ]:
telecom_cust.groupby('Contrato').Evasão.mean()

## Exercícios

a) Qual a taxa de evasão e não evasão entre os clientes idosos e não idosos? Plote um gráfico com esses dados.

b) Qual a distribuição de valores de cobrança mensal entre os clientes que evadiram?

c) Qual a distribuição de valores de cobrança mensal entre os clientes que **não** evadiram?

d) Qual a distribuição de valores de cobrança total entre os clientes que evadiram e não evadiram? Plote um gráfico comparando os dois,